In [0]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *

In [0]:
#dbutils.secrets.listScopes()

In [0]:
#dbutils.secrets.list(scope="databricksscope")

In [0]:
spark.conf.set("fs.azure.account.key.optumsadllssa.dfs.core.windows.net",dbutils.secrets.get(scope="databricksscope",key="adlskey"))

In [0]:
display(dbutils.fs.ls("abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net"))

path,name,size,modificationTime
abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net/Claims.csv,Claims.csv,5766,1716728851000
abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net/Patient_records.csv,Patient_records.csv,5110,1716718820000
abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net/disease.csv,disease.csv,1489,1716718820000
abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net/group.csv,group.csv,4390,1716718820000
abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net/hospital.csv,hospital.csv,1528,1716728950000
abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net/subgroup.csv,subgroup.csv,561,1716718820000
abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net/subscriber.csv,subscriber.csv,12061,1716718820000


In [0]:
data = spark.read.csv('abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net/subscriber.csv',header=True,inferSchema=True)

In [0]:
data.show(5,False)

+----------+----------+-----------+-----------+----------+------+--------------+-------+------------+--------+---------+--------+----------+----------+
|sub_id    |first_name|last_name  |Street     |Birth_date|Gender|Phone         |Country|City        |Zip Code|Subgrp_id|Elig_ind|eff_date  |term_date |
+----------+----------+-----------+-----------+----------+------+--------------+-------+------------+--------+---------+--------+----------+----------+
|SUBID10000|Harbir    |Vishwakarma|Baria Marg |1924-06-30|Female|+91 0112009318|India  |Rourkela    |767058  |S107     |Y       |1944-06-30|1954-01-14|
|SUBID10001|Brahmdev  |Sonkar     |Lala Marg  |1948-12-20|Female|+91 1727749552|India  |Tiruvottiyur|34639   |S105     |Y       |1968-12-20|1970-05-16|
|SUBID10002|Ujjawal   |Devi       |Mammen Zila|1980-04-16|Male  |+91 8547451606|India  |Berhampur   |914455  |S106     |N       |2000-04-16|2008-05-04|
|SUBID10003|Ballari   |Mishra     |Sahni Zila |1969-09-25|Female|+91 0106026841|India  |

In [0]:
data.count()

100

In [0]:
#data.select([count(when(isnan(c)|col(c).isNull(),c)).alias(c) for c in data.columns]).show()

In [0]:
data.groupby(data.columns).count().where("count>1").show()

+------+----------+---------+------+----------+------+-----+-------+----+--------+---------+--------+--------+---------+-----+
|sub_id|first_name|last_name|Street|Birth_date|Gender|Phone|Country|City|Zip Code|Subgrp_id|Elig_ind|eff_date|term_date|count|
+------+----------+---------+------+----------+------+-----+-------+----+--------+---------+--------+--------+---------+-----+
+------+----------+---------+------+----------+------+-----+-------+----+--------+---------+--------+--------+---------+-----+



In [0]:
data = data.dropDuplicates()

In [0]:
data = data.replace('NaN',None)

In [0]:
data = data.fillna({'Elig_ind':'N','first_name':"Guest/NA"})

In [0]:
data.show(5,False)

+----------+----------+-----------+------------+----------+------+--------------+-------+------------+--------+---------+--------+----------+----------+
|sub_id    |first_name|last_name  |Street      |Birth_date|Gender|Phone         |Country|City        |Zip Code|Subgrp_id|Elig_ind|eff_date  |term_date |
+----------+----------+-----------+------------+----------+------+--------------+-------+------------+--------+---------+--------+----------+----------+
|SUBID10001|Brahmdev  |Sonkar     |Lala Marg   |1948-12-20|Female|+91 1727749552|India  |Tiruvottiyur|34639   |S105     |Y       |1968-12-20|1970-05-16|
|SUBID10004|Devnath   |Srivastav  |Magar Zila  |1946-05-01|Female|+91 1868774631|India  |Bidhannagar |531742  |S110     |N       |1966-05-01|1970-12-09|
|SUBID10005|Atasi     |Seth       |Khatri Nagar|1967-10-02|Male  |+91 9747336855|India  |Amravati    |229062  |S104     |Y       |1987-10-02|1995-02-13|
|SUBID10003|Ballari   |Mishra     |Sahni Zila  |1969-09-25|Female|+91 0106026841|I

In [0]:
#data.select([count(when(isnan(c)|col(c).isNull(),c)).alias(c) for c in data.columns]).show()

In [0]:
data = data.drop('Phone')

In [0]:
data = data.withColumn("Subscriber_age",(months_between(current_date(),col('Birth_date'))/12).cast("int"))

In [0]:
#data.select([count(when(isnan(c)|col(c).isNull(),c)).alias(c) for c in data.columns]).show()

In [0]:
data.printSchema()

root
 |-- sub_id: string (nullable = true)
 |-- first_name: string (nullable = false)
 |-- last_name: string (nullable = true)
 |-- Street: string (nullable = true)
 |-- Birth_date: date (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zip Code: integer (nullable = true)
 |-- Subgrp_id: string (nullable = true)
 |-- Elig_ind: string (nullable = false)
 |-- eff_date: date (nullable = true)
 |-- term_date: date (nullable = true)
 |-- Subscriber_age: integer (nullable = true)



In [0]:
data = data.withColumn("Subscriber_age",(months_between(current_date(),col('Birth_date'))/12).cast("int"))

In [0]:
data.show()

+----------+----------+-----------+-----------------+----------+------+-------+--------------------+--------+---------+--------+----------+----------+--------------+
|    sub_id|first_name|  last_name|           Street|Birth_date|Gender|Country|                City|Zip Code|Subgrp_id|Elig_ind|  eff_date| term_date|Subscriber_age|
+----------+----------+-----------+-----------------+----------+------+-------+--------------------+--------+---------+--------+----------+----------+--------------+
|SUBID10001|  Brahmdev|     Sonkar|        Lala Marg|1948-12-20|Female|  India|        Tiruvottiyur|   34639|     S105|       Y|1968-12-20|1970-05-16|            75|
|SUBID10008|   Gurudas|      Gupta|      Sarin Nagar|1945-05-06|  Male|  India|           Kamarhati|  933226|     S103|       Y|1965-05-06|1970-09-16|            79|
|SUBID10019|  Guest/NA|     Maurya|     Sharaf Nagar|1924-11-09|Female|  India|            Jabalpur|  958538|     S104|       N|1944-11-09|1951-10-14|            99|
| SU

In [0]:
data = data.drop(col('Birth_date'))

In [0]:
data.select("*").where(col('Subgrp_id').isNull()).show()

+----------+----------+---------+---------+------+-------+--------+--------+---------+--------+----------+----------+--------------+
|    sub_id|first_name|last_name|   Street|Gender|Country|    City|Zip Code|Subgrp_id|Elig_ind|  eff_date| term_date|Subscriber_age|
+----------+----------+---------+---------+------+-------+--------+--------+---------+--------+----------+----------+--------------+
|SUBID10022|   Prakash|      Rao|   Sachar|Female|  India|Kottayam|  180680|     NULL|       N|1943-09-15|1948-10-19|           100|
|SUBID10049|   Paridhi|    Yadav|Sant Path|Female|  India|Jabalpur|  883754|     NULL|       N|1979-03-27|1985-06-01|            65|
+----------+----------+---------+---------+------+-------+--------+--------+---------+--------+----------+----------+--------------+



In [0]:
data = data.withColumn("Subgrp_id",when(col('sub_id')=="SUBID10022","S110").otherwise(col("Subgrp_id")))

In [0]:
data = data.withColumn("Subgrp_id",when(col('sub_id')=="SUBID10049","S107").otherwise(col("Subgrp_id")))

In [0]:
data.select("*").where(col('Subgrp_id').isNull()).show()

+------+----------+---------+------+------+-------+----+--------+---------+--------+--------+---------+--------------+
|sub_id|first_name|last_name|Street|Gender|Country|City|Zip Code|Subgrp_id|Elig_ind|eff_date|term_date|Subscriber_age|
+------+----------+---------+------+------+-------+----+--------+---------+--------+--------+---------+--------------+
+------+----------+---------+------+------+-------+----+--------+---------+--------+--------+---------+--------------+



In [0]:
#data.select([count(when(isnan(c)|col(c).isNull(),c)).alias(c) for c in data.columns]).show()

In [0]:
data.show()

+----------+----------+---------+-----------------+------+-------+----------------+--------+---------+--------+----------+----------+--------------+
|    sub_id|first_name|last_name|           Street|Gender|Country|            City|Zip Code|Subgrp_id|Elig_ind|  eff_date| term_date|Subscriber_age|
+----------+----------+---------+-----------------+------+-------+----------------+--------+---------+--------+----------+----------+--------------+
|SUBID10048|Chitranjan|   Mishra|      Madan Nagar|Female|  India|           Morbi|  945697|     S108|       N|1945-09-09|1952-06-21|            98|
|SUBID10001|  Brahmdev|   Sonkar|        Lala Marg|Female|  India|    Tiruvottiyur|   34639|     S105|       Y|1968-12-20|1970-05-16|            75|
|SUBID10095|   Ekaaksh|      Rai|      Bansal Ganj|Others|  India|Pimpri-Chinchwad|  158186|     S107|       N|1953-12-02|1960-07-29|            90|
|SUBID10066|  Guest/NA|     Seth|   Chaudhuri Marg|  Male|  India|        Kolhapur|  597470|     S102|    

In [0]:

output_container_path = "abfss://optumstagingdata@optumsadllssa.dfs.core.windows.net"
output_blob_folder = "optumstagingdata"
data.coalesce(1).write.format("csv").mode("overwrite").option("header","true").format("com.databricks.spark.csv").save(output_blob_folder)
files = dbutils.fs.ls(output_blob_folder)
output_file = [x for x in files if x.name.startswith("part-")]
dbutils.fs.mv(output_file[0].path,"%s/subscriberdatastaging.csv"%output_container_path)

True